# Interactive Visualizer (InVis)
This is the notebook where I'm currently developing the visualizer. I'm using Holoviews and Panel to construct the dashboard and implement the interactivity.

In [ ]:
import holoviews as hv
from holoviews import dim, opts, streams
from holoviews.selection import link_selections
import hvplot.pandas
import pandas as pd
from itertools import combinations
import numpy as np
from tqdm import trange, tqdm
import re
import panel as pn
import spatialpandas
import os
from bokeh.models import HoverTool
from classy import Class
import matplotlib.pyplot as plt
import copy

hv.extension('bokeh')
hv.Store.set_current_backend('bokeh')
pn.extension('tabulator')
pn.extension(loading_spinner='dots', loading_color='#00aa41', sizing_mode="stretch_width")
pn.extension()

## Functions

In [ ]:
# read in the .paramnames file and put it into list format
def load_params(filename):
    params_list = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            params = re.split(' \t ', line)
            params_list.append(params)
    return [item[0] for item in params_list]


# create a DataFrame with the chain files as rows and use a list of parameters as the column names
def load_data(filename, column_names):
    data = np.loadtxt(filename)
    df = pd.DataFrame(data[:,2:], columns=column_names)
    return df


# generate a scatter plot using the key dimensions from a holoviews.Dataset object, with the CLASS output displayed alongside it
def viz(data, data_classy_input, data_classy_CDM):
    # function for generating the scatter plot, given 2 dimensions as x and y axes, and an additional dimension to colormap
    # to the points on the plot. Also has an option to show or hide the colormap
    def plot_data(kdim1, kdim2, colordim, showcmap):
        if showcmap == True:
            cmapping = opts.Points(color=dim(colordim),
                colorbar=True,
                cmap='Viridis')
        else:
            cmapping = opts.Points(color='grey', colorbar=True)
        hover = HoverTool(tooltips=None)
        popts = opts.Points(
            toolbar='above',
            #line_color='black',
            #alpha=0.75, selection_alpha=1, nonselection_alpha=0.1,
            tools=[hover, 'box_select','lasso_select','tap'],
            size=7)
        points = hv.Points(data, kdims=[kdim1, kdim2]).opts(popts, cmapping)
        return points
    
    
    # setting Panel widgets for user interaction
    variables = data.columns.values.tolist()
    var1 = pn.widgets.Select(value=variables[0], name='Horizontal Axis', options=variables)
    var2 = pn.widgets.Select(value=variables[1], name='Vertical Axis', options=variables)
    cmap_var = pn.widgets.Select(value=variables[2], name='Colormapped Parameter', options=variables)
    cmap_option = pn.widgets.Checkbox(value=True, name='Show Colormap', align='end')
    
    # bind the widget values to the plotting function so it gets called every time the user interacts with the widget
    # call the bound plotting function inside a holoview DynamicMap object for interaction
    interactive_points = pn.bind(plot_data, kdim1=var1, kdim2=var2, colordim=cmap_var, showcmap=cmap_option)
    points_dmap = hv.DynamicMap(interactive_points, kdims=[]).opts(width=500, height=400, framewise=True)
    
    # define a stream to get a list of all the points the user has selected on the plot
    selection = streams.Selection1D(source=points_dmap)
    
    # function to generate a table of all the selected points
    def make_table(kdim1, kdim2, colordim):
        table = hv.DynamicMap(lambda index: hv.Table(data.iloc[index], kdims=[kdim1, kdim2, colordim]), streams=[selection])
        # formatting the table using plot hooks and a holoviews Options object
        def hook(plot, element):
            plot.handles['table'].autosize_mode = "none"
            for column in plot.handles['table'].columns:
                column.width = 100
            
        table_options = opts.Table(height=300, width=1000, hooks=[hook])
        return table.opts(table_options).relabel('Selected Points')
    
    
    # generate the table
    selected_table = pn.bind(make_table, kdim1=var1, kdim2=var2, colordim=cmap_var)
    
    #table_stream = streams.Selection1D(source=selected_table)
    
    # function to run CLASS on data from the selection. 
    # first create an empty plot to handle the null selection case
    empty_plot = hv.Curve(np.random.rand(0, 2))
    def run_class_on_selection(index):
        if not index:
            empty_pk = empty_plot.relabel('P(k) Residuals - no selection').opts(
                xlabel=r'$$k~[h/\mathrm{Mpc}]$$', 
                ylabel=r'$$(P(k)-P_{CDM}(k))/P_{CDM}(k)*100~[\%]$$')
            empty_cl_tt = empty_plot.relabel('Lensed Cl_TT Residuals - no selection').opts(
                xlabel=r"$$\ell$$", 
                ylabel=r"$$(C_{\ell}^{TT} - C_{\ell, CDM}^{TT})/C_{\ell, CDM}^{TT}*100~[\%]$$")
            empty_cl_ee = empty_plot.relabel('Lensed Cl_EE Residuals - no selection').opts(
                xlabel=r"$$\ell$$", 
                ylabel=r"$$(C_{\ell}^{EE} - C_{\ell, CDM}^{EE})/C_{\ell, CDM}^{EE}*100~[\%]$$")
            empty_layout = empty_pk + empty_cl_tt + empty_cl_ee            
            return empty_layout

        # the Selection1D stream returns an index number. index into the approprate dataframe and turn it into a dictionary for CLASS to read
        selection = data_classy_input.iloc[index]
        selection_CDM = data_classy_CDM.iloc[index]
        sel_dict_list = selection.to_dict('records')
        CDM_dict_list = selection_CDM.to_dict('records')
        
        # remove nuisance parameters and add some project-specific ones. in the future these will be defined by the user
#         for i in range(len(sel_dict_list)):
#             entries_to_remove1 = ('z_reio', 'A_s', 'sigma8', '100theta_s', 'A_cib_217', 'xi_sz_cib', 'A_sz', 'ps_A_100_100', 'ps_A_143_143', 'ps_A_143_217', 'ps_A_217_217', 'ksz_norm', 
#                                  'gal545_A_100', 'gal545_A_143', 'gal545_A_143_217', 'gal545_A_217', 'galf_TE_A_100', 'galf_TE_A_100_143', 'galf_TE_A_100_217', 'galf_TE_A_143', 'galf_TE_A_143_217', 
#                                  'galf_TE_A_217', 'calib_100T', 'calib_217T', 'A_planck' )
#             for j in entries_to_remove1:
#                 sel_dict_list[i].pop(j, None)

#             sel_dict_list[i]['omega_b'] = sel_dict_list[i]['omega_b']*1e-2
#             sel_dict_list[i]['sigma_dmeff'] = sel_dict_list[i]['sigma_dmeff']*1e-25
#             sel_dict_list[i]['h'] = (sel_dict_list[i].pop('H0'))*1e-2
#             sel_dict_list[i].update({"omega_cdm":1e-15, "npow_dmeff": 0, "Vrel_dmeff": 0, "dmeff_target": "baryons", "m_dmeff": 1e-3})

#         for k in range(len(sel_dict_list)):
#             params_CDM = copy.deepcopy(sel_dict_list[k])
#             entries_to_remove2 = ('sigma_dmeff', 'npow_dmeff', 'Vrel_dmeff', 'dmeff_target', 'm_dmeff')
#             for l in entries_to_remove2:
#                 params_CDM.pop(l, None)
#             params_CDM['omega_cdm'] = params_CDM.pop('omega_dmeff')
#             CDM_dict_list.append(params_CDM)

        # run class on the user's selection
        cosmo = Class()
        cosmo.set(sel_dict_list[0])
        cosmo.set({'output':'mPk, tCl, pCl, lCl','P_k_max_1/Mpc':3.0, 'lensing':'yes'})
        cosmo.compute()

        # set variables for matter power spectrum and lensed CMB angular power spectra
        kk = np.logspace(-4,np.log10(3),1000)
        Pk1 = []
        h = cosmo.h()
        for k in kk:
            Pk1.append(cosmo.pk(k*h,0.)*h**3)

        l = np.array(range(2,2501))
        factor = l*(l+1)/(2*np.pi)
        lensed_cl = cosmo.lensed_cl(2500)
        
        # run CLASS on the user's selection using the CDM model
        cosmo_CDM = Class()
        cosmo_CDM.set(CDM_dict_list[0])
        cosmo_CDM.set({'output':'mPk, tCl, pCl, lCl','P_k_max_1/Mpc':3.0, 'lensing':'yes'})
        cosmo_CDM.compute()

        # set variables for CDM observables
        Pk_CDM = []
        h = cosmo_CDM.h()
        for k in kk:
            Pk_CDM.append(cosmo_CDM.pk(k*h,0.)*h**3)

        lensed_cl_CDM = cosmo_CDM.lensed_cl(2500)

        # compute residuals
        pk_residuals = (np.array(Pk1) - np.array(Pk_CDM))/np.array(Pk_CDM)*100
        cl_tt_residuals = (lensed_cl['tt'][2:] - lensed_cl_CDM['tt'][2:])/(lensed_cl_CDM['tt'][2:])*100
        cl_ee_residuals = (lensed_cl['ee'][2:] - lensed_cl_CDM['ee'][2:])/(lensed_cl_CDM['ee'][2:])*100

        plot_pk_residuals = hv.Curve((kk, pk_residuals)).relabel('P(k) Residuals').opts(
            xlabel=r'$$k~[h/\mathrm{Mpc}]$$', 
            ylabel=r'$$(P(k)-P_{CDM}(k))/P_{CDM}(k)*100~[\%]$$')

        plot_cl_tt_residuals = hv.Curve((l,cl_tt_residuals)).relabel('Cl_TT Residuals').opts(
            xlabel=r"$$\ell$$", 
            ylabel=r"$$(C_{\ell}^{TT}-C_{\ell, CDM}^{TT})/C_{\ell, CDM}^{TT}*100~[\%]$$")

        plot_cl_ee_residuals = hv.Curve((l,cl_ee_residuals)).relabel('Cl_EE Residuals').opts(
            xlabel=r"$$\ell$$", 
            ylabel=r"$$(C_{\ell}^{EE}-C_{\ell, CDM}^{EE})/C_{\ell, CDM}^{EE}*100~[\%]$$")
        
        layout = (plot_pk_residuals + plot_cl_tt_residuals + plot_cl_ee_residuals)
        return layout
    
    
    classy_output = hv.DynamicMap(run_class_on_selection, streams=[selection]).opts(
        opts.Curve(color='black', logx=True, width=500, height=400, padding=0.1, framewise=True),
        opts.Layout(shared_axes=False))
    
    classy_output_pane = pn.panel(classy_output)
    # pn.param.set_values(classy_output_pane, loading=True)
    
    # put it all together using Panel
    points_display = pn.Column(pn.Row(var1, var2, cmap_var, cmap_option), pn.Row(points_dmap, selected_table))
    dashboard = pn.Column(points_display, classy_output_pane)
    return dashboard

## Visualizing
Now we just need to load in our data, convert it into a format that CLASS accepts, and call `viz` to produce the dashboard.

In [ ]:
if __name__=='__main__':
    # Read in data
    param_names = load_params('/home/jswen/dev/inviz/data/test_IDM_n_0/2022-05-04_75000_.paramnames')
    df = pd.DataFrame(columns=param_names)
    for i in trange(1,57):
        temp = load_data('/home/jswen/dev/inviz/data/test_IDM_n_0/2022-05-04_75000__{}.txt'.format(i), column_names=param_names)
        df = pd.concat([df,temp]).reset_index(drop=True)

In [ ]:
# prepare data for CLASS computation
# remove nuisance parameters
classy_input = df.drop(columns=['z_reio', 'A_s', 'sigma8', '100theta_s', 'A_cib_217', 'xi_sz_cib', 'A_sz', 'ps_A_100_100', 'ps_A_143_143', 'ps_A_143_217', 'ps_A_217_217', 'ksz_norm', 
                 'gal545_A_100', 'gal545_A_143', 'gal545_A_143_217', 'gal545_A_217', 'galf_TE_A_100', 'galf_TE_A_100_143', 'galf_TE_A_100_217', 'galf_TE_A_143', 'galf_TE_A_143_217', 
                 'galf_TE_A_217', 'calib_100T', 'calib_217T', 'A_planck'])
classy_input['omega_b'] = df['omega_b'] * 1e-2
classy_input['sigma_dmeff'] = df['sigma_dmeff'] * 1e-25
classy_input = classy_input.rename(columns={'H0':'h'})
classy_input['h'] = classy_input['h'] * 1e-2
classy_input['omega_cdm'] = 1e-15
classy_input['npow_dmeff'] = 0.0
classy_input['Vrel_dmeff'] = 0.0
classy_input['dmeff_target'] = 'baryons'
classy_input['m_dmeff'] = 1e-3

# format for CDM version
classy_CDM = classy_input.drop(columns=['sigma_dmeff', 'omega_cdm', 'npow_dmeff', 'Vrel_dmeff', 'dmeff_target', 'm_dmeff'])
classy_CDM = classy_CDM.rename(columns={'omega_dmeff':'omega_cdm'})

# slice for fast computation
classy_input_slice = classy_input[::500].reset_index(drop=True)
classy_CDM_slice = classy_CDM[::500].reset_index(drop=True)
df_slice = df[::500].reset_index(drop=True)

### The Viz Function

The viz function has three arguments:
- __data__: the data you want shown as a scatter plot
- __data_classy_input__: the data that you've prepared to feed into CLASS
- __data_classy_CDM__: the CDM version of that data

Be aware that if you pass data with categorical values (i.e. strings) in the first argument, the visualizer will break. It's fine to do so for the CLASS arguments though.

In [ ]:
viz(df_slice, classy_input_slice, classy_CDM_slice)

## WIP
The below cells are being used to further develop the visualizer.

In [ ]:
test_params = {'omega_b': 2.268118e-2, 
               'omega_dmeff': 0.1221163, 
               'ln10^{10}A_s': 3.048006, 
               'n_s': 0.9670508, 
               'tau_reio': 0.05659527, 
               'sigma_dmeff': 1.193e-25, 
               'Omega_Lambda': 0.6772064, 
               'YHe': 0.2479716, 
               'h': 67.13298e-2,  
               'omega_cdm': 1e-15, 
               'npow_dmeff': 0, 
               'Vrel_dmeff': 0, 
               'dmeff_target': 'baryons', 
               'm_dmeff': 0.001}
test_params_CDM = {'omega_b': 2.268118e-2, 
                   'omega_cdm': 0.1221163, 
                   'ln10^{10}A_s': 3.048006, 
                   'n_s': 0.9670508, 
                   'tau_reio': 0.05659527, 
                   'Omega_Lambda': 0.6772064, 
                   'YHe': 0.2479716, 
                   'h': 67.13298e-2}

In [ ]:
cosmo = Class()
cosmo.set(test_params)
cosmo.set({'output':'mPk, tCl, pCl, lCl','P_k_max_1/Mpc':3.0, 'lensing':'yes'})
cosmo.compute()

kk = np.logspace(-4,np.log10(3),1000)
Pk1 = []
h = cosmo.h()
for k in kk:
    Pk1.append(cosmo.pk(k*h,0.)*h**3)

l = np.array(range(2,2501))
factor = l*(l+1)/(2*np.pi)
lensed_cl = cosmo.lensed_cl(2500)

In [ ]:
cosmo_CDM = Class()
cosmo_CDM.set(test_params_CDM)
cosmo_CDM.set({'output':'mPk, tCl, pCl, lCl','P_k_max_1/Mpc':3.0, 'lensing':'yes'})
cosmo_CDM.compute()

Pk_CDM = []
h = cosmo_CDM.h()
for k in kk:
    Pk_CDM.append(cosmo_CDM.pk(k*h,0.)*h**3)

lensed_cl_CDM = cosmo_CDM.lensed_cl(2500)

In [ ]:
fig_PK = plt.loglog(kk, Pk1)
fig_PKCDM = plt.loglog(kk, Pk_CDM)

In [ ]:
fig_TT = plt.loglog(l, factor*lensed_cl['tt'][2:])
fig_TTCDM = plt.loglog(l, factor*lensed_cl_CDM['tt'][2:])

In [ ]:
# compute residuals and plot them
pk_residuals = (np.array(Pk1) - np.array(Pk_CDM))/np.array(Pk_CDM)*100
cl_tt_residuals = (lensed_cl['tt'][2:] - lensed_cl_CDM['tt'][2:])/(lensed_cl_CDM['tt'][2:])*100
cl_ee_residuals = (lensed_cl['ee'][2:] - lensed_cl_CDM['ee'][2:])/(lensed_cl_CDM['ee'][2:])*100

In [ ]:
plot_pk_residuals = hv.Curve((kk, pk_residuals)).relabel('P(k) Residuals').opts(
    xlabel=r'$$k~[h/\mathrm{Mpc}]$$', 
    ylabel=r'$$(P(k)-P_{CDM}(k))/P_{CDM}(k)*100~[\%]$$'
)

plot_cl_tt_residuals = hv.Curve((l, cl_tt_residuals)).relabel('Cl_TT Residuals').opts(
    xlabel=r"$$\ell$$", 
    ylabel=r"$$(C_{\ell}^{TT} - C_{\ell, CDM}^{TT})/C_{\ell, CDM}^{TT}*100~[\%]$$"
)

plot_cl_ee_residuals = hv.Curve((l, cl_ee_residuals)).relabel('Cl_EE Residuals').opts(
    xlabel=r"$$\ell$$", 
    ylabel=r"$$(C_{\ell}^{EE} - C_{\ell, CDM}^{EE})/C_{\ell, CDM}^{EE}*100~[\%]$$"
)


layout = (plot_pk_residuals + plot_cl_tt_residuals + plot_cl_ee_residuals)
layout = layout.opts(opts.Curve(logx=True, width=500, height=400, padding=0.1, fontscale=1.1), opts.Layout(shared_axes=False))
layout

In [ ]:
def plot_scatter_table(data, params, plots):
    # kwargs:
    # data: a pandas DataFrame
    # params: list of parameters that correspond to column names in the DataFrame
    # plots: the number of plots to display
    
    # generate a list of all pairs of the parameters
    pairs = [list(comb) for comb in combinations(params, 2)]
    
    # create linked selections
    ls = link_selections.instance()
    
    layout = hv.Layout()
    
    for param_a, param_b in pairs[:plots]:
        # vdims = [e for e in params if e not in (param_a, param_b)]
        # ^^^ uncomment the above and add ", vdims" to the argument of hv.Dataset if desired
        ds = hv.Dataset(data, [param_a, param_b])
        pts = hv.Points(ds).opts(
            opts.Points(color='black', size=2))
        bivar = hv.Bivariate(data[[param_a,param_b]].values, [param_a,param_b], []).opts(
            opts.Bivariate(bandwidth=0.5,
                           cut=0,cmap="blues",
                           levels=5,
                           colorbar=False,
                           show_legend=False,
                           filled=True,
                           toolbar='above',
                           width=350,
                           alpha=0.75))
        layout += (ls(pts)*bivar).opts(width=300, height=300)
    
    layout = layout.cols(4)
    
    table = pn.widgets.Tabulator(data[params], disabled=True)
    selection = pn.widgets.Tabulator(disabled=True)
    try:
        table.selection = ls.selection_expr
    except ValueError:
        table.selection = []
    
    button_show_selection = pn.widgets.Button(name='Show selected rows only', button_type='primary', width_policy='auto')
    button_download_selection = pn.widgets.Button(name='Download selections as CSV', button_type='primary', width_policy='auto')
    button_reset = pn.widgets.Button(name='Reset', width_policy='auto')
    
    def show_selection(event):
        selection.value = table.selected_dataframe.sort_index()
    
    def download_selection(event):
        os.makedirs('data/selections', exist_ok=True)
        selection_df_sorted = table.selected_dataframe.sort_index()
        selection_df_sorted.to_csv('data/selections/selected_output.csv', index=False)

    def reset(event):
        table.selection = []
        selection.value = table.selected_dataframe

    button_show_selection.on_click(show_selection)
    button_download_selection.on_click(download_selection)
    button_reset.on_click(reset)
    dashboard = pn.Row(table, pn.Column(pn.Row(button_show_selection, button_download_selection, button_reset), selection))
    params = pn.Param(ls, parameters=['selection_mode'])
    scatter_table = pn.Column(params, pn.Row(layout), dashboard)
    return scatter_table

In [ ]:
params = ['omega_b', 'omega_dmeff', 'n_s', 'tau_reio', 'sigma_dmeff', 'H0', 'A_s', 'sigma8']
viz = plot_scatter_table(new_df, params, plots=4)
viz

In [ ]:
menu_items = list(ds.data)
menu_button = pn.widgets.MenuButton(name='Parameter', items=menu_items, button_type='primary')
text = pn.widgets.TextInput(value='Ready')

def b(event):
    text.value = f'Clicked menu item: "{event.new}"'
    
menu_button.on_click(b)

pn.Row(menu_button, text, height=300)

In [ ]:
ds = hv.Dataset(new_df, ['omega_b', 'sigma_dmeff'])
vdims = ['omega_b', 'omega_dmeff', 'ln10^{10}A_s', 'n_s', 'tau_reio', 'sigma_dmeff', 'Omega_Lambda', 'H0']
layout = hv.Layout()
for vdim in vdims:
    if vdim != ds.kdims[0].name and vdim != ds.kdims[1].name:
        pts = hv.Points(ds).opts(color=dim(vdim),
            colorbar=True,
            cmap='Magma',
            alpha=0.5, selection_alpha=1, nonselection_alpha=0.1,
            tools=['box_select','lasso_select','tap'],
            size=5,
            width=350,
            height=250).relabel(vdim)
        layout += pts

layout.opts(shared_axes=False).cols(3)

In [ ]:
def plot_data(kdim1, kdim2, colordim, showcmap):  
    popts = opts.Points(
        toolbar='above',
        line_color='black',
        #alpha=0.5, selection_alpha=1, nonselection_alpha=0.1,
        tools=['box_select','lasso_select','tap'],
        size=7)
    
    if classy_input_slice[colordim].dtype == object:
        if showcmap == False:
            cmapping = opts.Points(color='grey')
        cmapping = opts.Points(color=dim(colordim), cmap='Category20')
        categorical_points = hv.Points(classy_input_slice, kdims=[kdim1, kdim2]).opts(popts, cmapping)
        return categorical_points
    else:
        if showcmap == False:
            cmapping = opts.Points(color='grey')
        cmapping = opts.Points(color=dim(colordim), cmap='Viridis', colorbar=True)
        numerical_points = hv.Points(classy_input_slice, kdims=[kdim1, kdim2]).opts(popts, cmapping)
        return numerical_points
        

def plot_dist(kdim1):
    dist = hv.Distribution(new_df, kdim1).opts(
        alpha=1,
        color='blue')
    return dist

def plot_bivar(kdim1, kdim2):
    bivar = hv.Bivariate(new_df, kdims=[kdim1, kdim2]).opts(
        cmap='blues',
        bandwidth=1,
        cut=0,
        levels=5,
        filled=True)
    return bivar

def choose_dist(kdim1, kdim2):
    if kdim1 == kdim2:
        return hv.DynamicMap(plot_dist, kdim1=kdim1).opts(width=500, height=400, framewise=True)
    else:
        return hv.DynamicMap(plot_bivar, kdim1=kdim1, kdim2=kdim2).opts(width=500, height=400, framewise=True)

In [ ]:
variables = classy_input_slice.columns.values.tolist()
var1 = pn.widgets.Select(value=variables[0], name='Horizontal Axis', options=variables)
var2 = pn.widgets.Select(value=variables[1], name='Vertical Axis', options=variables)
cmap_var = pn.widgets.Select(value='omega_b', name='Colormapped Parameter', options=variables)
cmap_option = pn.widgets.Checkbox(value=False, name='Show Colormap', align='end')

In [ ]:
interactive_points = pn.bind(plot_data, kdim1=var1, kdim2=var2, colordim=cmap_var, showcmap=cmap_option)
interactive_dist = pn.bind(choose_dist, kdim1=var1, kdim2=var2)
dmap_points = hv.DynamicMap(interactive_points).opts(width=500, height=400, framewise=True)
app = pn.Column(pn.Row(var1, var2, cmap_var, cmap_option, width = 700), dmap_points)
app

In [ ]:
choose_dist('omega_b', 'omega_dmeff')

In [ ]:
table = hv.Table(new_df, kdims=['omega_b', 'omega_dmeff', 'sigma_dmeff'])
table_sel = streams.Selection1D(source=table)

In [ ]:
table_sel.contents